# Libraries

In [ ]:
import gym
import os
import time
import matplotlib.pyplot as plt
from pyswip import Prolog
from utils import create_level, define_reward, process_state, perform_action, show_match,display_inventory,parse_predicate,extract_monsters
from nle import nethack
from utilsbattle import get_money_location, get_stair_location, get_target_location
#from shop_generator import creashop

#import pyswip as psp
import run
import random

# Simulation parameters

In [ ]:
NUM_EPISODES = 5
MAX_STEPS = 100
PATH = 'kb.pl'
RISK_THRESHOLD=1# when the agent feels it's not healthy anymore
MONEY_THRESHOLD=1
MONEY=40
ARROWS= 2
ARROWS_THRESHOLD=4
BATTLE_PATH = 'kb-battle.pl'

Initialize the knowledge base.

In [ ]:
KB = Prolog()
KB.consult(PATH)

#KB.consult(BATTLE_PATH)


#loading initial agent paramters into the knowledge base
KB.asserta(f"health_threshold({RISK_THRESHOLD})")
KB.asserta(f"money_threshold({MONEY_THRESHOLD})")
KB.asserta(f"money({MONEY})")
KB.asserta(f"arrows({ARROWS})")
KB.asserta(f"arrows_threshold({ARROWS_THRESHOLD})")

KB.retractall('onPlan(_)')





#### Main code
- Perform `NUM_EPISODES` experiences in the environment.
- Use `Prolog` to define the axioms and choose the action to perform.
- The main goal is to _reach and eat_ the `apple`.

In [ ]:
rewards = []
action=None

ran=run.run()
for episode in range(1):
    des_file,room_type = run.completemapdes(1,5,'maze')
    #des_file=ran.getdes()
    #room_type=ran.gettype()
    actions=tuple(nethack.CompassDirection )+(nethack.Command.PICKUP,nethack.Command.PAY,nethack.Command.WIELD,nethack.Command.SIT,nethack.Command.QUAFF)
    env = gym.make("MiniHack-Skill-Custom-v0",
                    character="Agent-cav-hum-neu-mal",
                    observation_keys=('screen_descriptions','inv_strs','blstats','message','pixel','chars','inv_oclasses','glyphs'),
                    wizard = True,
                    options =(
                        "color",  # Display color for different monsters, objects, etc
                        "showexp",  # Display the experience points on the status line
                        "nobones",  # Disallow saving and loading bones files
                        "nolegacy",  # Not display an introductory message when starting the game
                        "nocmdassist",  # No command assistance
                        "disclose:+i +a +v +g +c +o",  # End of game prompt replies
                        "runmode:teleport",  # Update the map after movement has finished
                        "mention_walls",  # Give feedback when walking against a wall
                        "nosparkle",  # Not display sparkly effect for resisted magical attacks
                        "showscore",  # Shows approximate accumulated score on the bottom line
                        "race:hum","gender:mal","align:neu","role:cav",
                        "!autopickup",
                        "pettype:none",
                    ),
                    des_file=des_file,
               )
    obs=env.reset()
    #print(des_file)
    # count the number of steps of the current episode
    steps = 0
    # store the cumulative reward
    reward = 0.0
    # collect obs['pixel'] to visualize
    ep_states = []
    #agent starts in the shop
    in_battle = False
    #agent start with no plan for battle
    planned_actions = False


    obs = env.reset()

    #process level information
    if any(map(lambda x:x==room_type,['mazeroom','minibossroom'])):
        KB.asserta("corridors")
    monsters=extract_monsters(des_file)
    goal_x,goal_y=-1,-1
    level_heigth=len(obs['screen_descriptions'])
    level_width=len(obs['screen_descriptions'][0])
    for i in range(level_heigth):
        for j in range(level_width):
            objs = bytes(obs['screen_descriptions'][i][j]).decode('utf-8').rstrip('\x00')
            if 'closed door' in objs and i > goal_x:
                goal_x,goal_y=i,j
    KB.asserta(f'battlefield_start({goal_x},{goal_y})')
        
    ep_states.append(obs['pixel'])
    done = False
    # Main loop
    while not done and steps < MAX_STEPS:

        process_state(obs, KB, in_battle = in_battle)

        #check if process state has seen that we are in the position to start battle
        if (not in_battle) and len(list(KB.query('battle_begin'))) > 0 : #if yes switch to battle mode
           
            in_battle = True
            #retrieve money amount
            
            game_map = obs['chars']
            game = obs['pixel']

            #retrieve map informations
            target = get_target_location(game_map)
            stair_des_pos = get_stair_location(des_file)

            #when looking at thes to get information, the position of the items in the real envirment is translated by a certain vector
            delta_x, delta_y = (target[0]-stair_des_pos[0],target[1]-stair_des_pos[1])
            gold_pos_list = get_money_location(des_file)

            gold_pos_list = [(x+delta_x,y+delta_y)for (_,(x,y)) in gold_pos_list] #modify here if interested in gold amount on ground

            #tell to KB gold positions
            for (x,y) in gold_pos_list :
                KB.asserta(f'position(gold,_,{x},{y})')
            #tell to KB stairs down location
            print('ARRIVO GIUSTO')
            KB.asserta(f'position(stairs,_,{target[0]},{target[1]})')

        #only done to identify monster position on 
        if not in_battle :
            for monster_name,pos_x,pos_y in monsters:
                KB.asserta(f"position(enemy,\'{monster_name}\',{pos_x},{pos_y})")
       
        try:
            actions=list(KB.query('action(X)'))
            action = actions[0]
            action = action['X']
        except Exception as e:
            #print("no action available")
            #print(list(KB.query("position(object,Type,R,C)")))
            print(e)
            action = None
        # Perform the action in the environment
            
        if action:
            obs, reward, done, info ,planned_actions = perform_action(action, env,KB,planned_actions,obs)
            ep_states.append(obs['pixel'])
            env.render()
        else:
            break
        steps += 1
    if steps == MAX_STEPS:
        print("reached maximum amount of steps")
    env.render()
    # Display game with interface
    #show_match(ep_states)
    rewards.append(reward)
    
    # reset the environment and retract axioms that may cause errors
    #print(list(KB.query('battle_begin')))
    print(list(KB.query('position(stairs,_,X,Y)')))

    obs = env.reset()
    KB.retractall('stepping_on(agent,_,_)')
    KB.retractall('position(_,_,_,_)')
    KB.retractall('shopping_done')
    KB.retractall('corridors')
    KB.retractall('battle_begin')
    KB.retractall(f'battlefield_start(_,_)')
    ran.nextlevel()


print(f'After {NUM_EPISODES} episodes, mean return is {sum(rewards)/NUM_EPISODES}')
print("The rewards of the episodes are:", rewards)

In [ ]:
des_file,room_type = run.completemapdes(1,5,'maze')
env = gym.make("MiniHack-Skill-Custom-v0",
                    character="Agent-cav-hum-neu-mal",
                    observation_keys=('screen_descriptions','inv_strs','blstats','message','pixel','chars','inv_oclasses','glyphs'),
                    wizard = True,
                    options =(
    "color",  # Display color for different monsters, objects, etc
    "showexp",  # Display the experience points on the status line
    "nobones",  # Disallow saving and loading bones files
    "nolegacy",  # Not display an introductory message when starting the game
    "nocmdassist",  # No command assistance
    "disclose:+i +a +v +g +c +o",  # End of game prompt replies
    "runmode:teleport",  # Update the map after movement has finished
    "mention_walls",  # Give feedback when walking against a wall
    "nosparkle",  # Not display sparkly effect for resisted magical attacks
    "showscore",  # Shows approximate accumulated score on the bottom line
    "race:hum","gender:mal","align:neu","role:cav",
    "!autopickup",
    "pettype:none",
),
                    des_file=des_file,
               )
obs=env.reset(wizkit_items = ['16 scroll of fire','13 potion of confusion','unicorn horn','healing potion'])
env.render()

In [ ]:
for item in obs['inv_strs']:
    item = bytes(item).decode('utf-8').rstrip('\x00')
    print(item)

In [ ]:
for idx,action in enumerate(env.actions):
    print(idx,action)

In [ ]:
#27 to drop
#69 to take off
#52 quaff
#54 read
#env.step(54)
#env.render()
#env.step(env.actions.index(ord('f')))
#you feel less confused now.
env.step(env.actions.index(ord('h')))
env.render()

In [ ]:
def set_health(env, target_health) :
    #psedocodice, scrivo dopo
    #until not confuso -> drink potion of confusion
    #read scroll until healt is right
    #use unicorn horn
    #drop potions
    #drop scroll
    pass